<a href="https://colab.research.google.com/github/lauberto/uni_mydirectory/blob/master/compling/homework/HW8_andrea_grillandi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
!pip install pymorphy2

In [0]:
import json, os
from google.colab import drive
import pandas as pd
from nltk.corpus import stopwords
import numpy as np
from pymorphy2 import MorphAnalyzer
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
morph = MorphAnalyzer()
stops = set(stopwords.words('russian'))

In [0]:
from string import punctuation
from nltk.corpus import stopwords
punct = punctuation+'«»—…“”*№–'
stops = set(stopwords.words('russian'))

In [0]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
PATH_TO_DATA = os.path.join(os.getcwd(), 'drive', 'My Drive', 'ru_kw_eval_datasets', 'data')

In [0]:
os.listdir(PATH_TO_DATA)

['cyberleninka_4.jsonlines.zip',
 'russia_today_7.jsonlines.zip',
 'cyberleninka_0.jsonlines.zip',
 'cyberleninka_1.jsonlines.zip',
 'cyberleninka_2.jsonlines.zip',
 'cyberleninka_3.jsonlines.zip',
 'habrahabr_0.jsonlines.zip',
 'habrahabr_1.jsonlines.zip',
 'habrahabr_2.jsonlines.zip',
 'habrahabr_3.jsonlines.zip',
 'ng_0.jsonlines.zip',
 'ng_1.jsonlines.zip',
 'russia_today_0.jsonlines.zip',
 'russia_today_1.jsonlines.zip',
 'russia_today_2.jsonlines.zip',
 'russia_today_3.jsonlines.zip',
 'russia_today_4.jsonlines.zip',
 'russia_today_5.jsonlines.zip',
 'russia_today_6.jsonlines.zip']

In [0]:
files = [os.path.join(PATH_TO_DATA, file) for file in os.listdir(PATH_TO_DATA)]

In [0]:
data = pd.concat([pd.read_json(file, lines=True) for file in files], axis=0, ignore_index=True)

In [0]:
data.shape

(17266, 6)

In [0]:
def evaluate(true_kws, predicted_kws):
    assert len(true_kws) == len(predicted_kws)
    
    precisions = []
    recalls = []
    f1s = []
    jaccards = []
    
    for i in range(len(true_kws)):
        
        true_kw = set(true_kws[i])
        if predicted_kws[i]:
          predicted_kw = set(predicted_kws[i])
        else:
          predicted_kw=set()
        
        tp = len(true_kw & predicted_kw)
        union = len(true_kw | predicted_kw)
        fp = len(predicted_kw - true_kw)
        fn = len(true_kw - predicted_kw)
        
        if (tp+fp) == 0:
            prec = 0
        else:
            prec = tp / (tp + fp)
        
        if (tp+fn) == 0:
            rec = 0
        else:
            rec = tp / (tp + fn)
        if (prec+rec) == 0:
            f1 = 0
        else:
            f1 = (2*(prec*rec))/(prec+rec)
            
        jac = tp / union
        
        precisions.append(prec)
        recalls.append(rec)
        f1s.append(f1)
        jaccards.append(jac)
    print('Precision - ', round(np.mean(precisions), 2))
    print('Recall - ', round(np.mean(recalls), 2))
    print('F1 - ', round(np.mean(f1s), 2))
    print('Jaccard - ', round(np.mean(jaccards), 2))

In [0]:
evaluate(data['keywords'], data['keywords'])

Precision -  1.0
Recall -  1.0
F1 -  1.0
Jaccard -  1.0


### Stemming

In [0]:
from nltk.stem.snowball import SnowballStemmer

In [0]:
stemmer = SnowballStemmer('russian')

In [0]:
stemmer.stem("собака")

'собак'

In [0]:
def stemming(text):
    words = [word.strip(punct) for word in text.lower().split()]
    words = [stemmer.stem(word) for word in words if word and word not in stops]
    return words

In [0]:
data['content_stem'] = data['content'].apply(stemming)

In [0]:
data['keywords_str'] = data['keywords'].apply(' '.join)

In [0]:
data['keywords_stem'] = data['keywords_str'].apply(stemming)

In [0]:
evaluate(data['keywords_stem'], data['content_stem'].apply(lambda x: [x[0] for x in Counter(x).most_common(10)]))

Precision -  0.23
Recall -  0.27
F1 -  0.23
Jaccard -  0.14


Стемминг немного улучшает метрики, однако надо учитывать то, что возможно разные слова одинаково стеммируются. При этом, полученные результаты могут чуть отличаться от реальных. Но, поскольку итоговая задача - понять о чем текст, вряд ли это сильно разрушает наш цель.

### Стемминг + Tfidf
Попробуем сложить с векторизацией Tfidf

In [0]:
data['content_stem_str'] = data['content_stem'].apply(' '.join)

In [0]:
tfidf = TfidfVectorizer(ngram_range=(1,3), min_df=5, max_df=0.7, max_features=750)

In [0]:
tfidf.fit(data['content_stem_str'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=0.7, max_features=750,
                min_df=5, ngram_range=(1, 3), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [0]:
id2word = {i:word for i, word in enumerate(tfidf.get_feature_names())}

In [0]:
texts_vectors = tfidf.transform(data['content_stem_str'])

In [0]:
def kw_extraction(texts_vectors):
  keywords=[]

  for row in range(texts_vectors.shape[0]):
    row_data = texts_vectors.getrow(row)
    top_inds = row_data.toarray().argsort()[0,:-11:-1]
    keywords.append([id2word[w] for w in top_inds])
  return keywords

In [0]:
evaluate(data['keywords_stem'], kw_extraction(texts_vectors))

Precision -  0.13
Recall -  0.13
F1 -  0.12
Jaccard -  0.07


Результат никак не улучшился, сейчас попробуем с теми же параметрами векторизации, но с чисто лемматизированными текстами.

In [0]:
def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0] for word in words if word and word not in stops]
    words = [word.normal_form for word in words if word.tag.POS == 'NOUN']

    return words

In [0]:
data['content_norm'] = data['content'].apply(normalize)

In [0]:
# data['content_norm_str'] = data['content_norm'].apply(' '.join)

In [0]:
tfidf.fit(data['content_norm_str'])

In [0]:
id2word = {i:word for i, word in enumerate(tfidf.get_feature_names())}

In [0]:
texts_vectors = tfidf.transform(data['content_norm_str'])

In [0]:
evaluate(data['keywords'], kw_extraction(texts_vectors))

### Пересекающиеся окна и TextRank

In [0]:
from itertools import combinations

In [0]:
def get_kws(text, window_size=5, random_p=0.1):

    vocab = set(text)
    word2id = {w:i for i, w in enumerate(vocab)}
    id2word = {i:w for i, w in enumerate(vocab)}
    ids = [word2id[word] for word in text]

    m = np.zeros((len(vocab), len(vocab)))

    for i in range(0, len(ids), 2): # here is the new step
        window = ids[i:i+window_size]
        
        for j, k in combinations(window, 2):
            m[j][k] += 1
            m[k][j] += 1
    
    for i in range(m.shape[0]):
        s = np.sum(m[i])
        if not s:
            continue
        m[i] /= s
    
    pr = np.full((len(vocab), 1), 1)
    d = 0.85 # d - это dumping factor,  which has the role of integrating into the model the probability of jumping from a given vertex to another random vertex in the graph
    
    for i in range(10):
      pr = 0.15 + 0.85 * np.dot(m, pr)
    pr = pr[:,0]
    return [id2word[i] for i in pr.argsort()[:-11:-1]] # берем топ10

In [0]:
%%time
keywords_tr = data['content_norm'].apply(lambda x: get_kws(x, 5))

CPU times: user 3min 47s, sys: 2min 33s, total: 6min 21s
Wall time: 3min 13s


In [0]:
keywords_tr

0        [редактор, отношение, управление, арк, народ, ...
1        [настоящее, предбайкалье, качество, заместител...
2        [вычисление, порядок, автор, обзор, тип, центр...
3        [век, молодая, разочарование, система, рыба, д...
4        [право, лишение, слово, лишенец, провинция, ис...
                               ...                        
17261    [самолёт, комиссия, заместитель, руслан, самол...
17262    [рынок, шутка, движение, лидер, криптоинвестор...
17263    [звезда, создание, цель, облако, голограмма, в...
17264    [состав, школа, количество, чемпионат, секрет,...
17265    [объединение, политик, митинг, гражданин, сутк...
Name: content_norm, Length: 17266, dtype: object

In [0]:
evaluate(data['keywords'], keywords_tr)

Precision -  0.02
Recall -  0.03
F1 -  0.02
Jaccard -  0.01


Видимо, этот алгоритм очень плохо работает, возможно надо было отфилтрировать слова по частоте или по POS.

In [0]:
def get_kws_old(text, top=5, window_size=5, random_p=0.1):

    vocab = set(text)
    if len(vocab)!=0:
      word2id = {w:i for i, w in enumerate(vocab)}
      id2word = {i:w for i, w in enumerate(vocab)}
      ids = [word2id[word] for word in text]

      m = np.zeros((len(vocab), len(vocab)))

      for i in range(0, len(ids), 2):
          window = ids[i:i+window_size]
          
          for j, k in combinations(window, 2): 
              m[j][k] += 1
              m[k][j] += 1
      
      for i in range(m.shape[0]):
          s = np.sum(m[i])
          if not s:
              continue
          m[i] /= s
      
      c = Counter()

      if len(vocab)!=0:
        n = np.random.choice(len(vocab))
      else:
        n = np.random.choice(1000)
      for i in range(500): 
          
          go_random = np.random.choice([0, 1], p=[1-random_p, random_p])
          
          if go_random:
              n = np.random.choice(len(vocab))
          
          n = take_step(n, m)
          c.update([n])
      
      return [id2word[i] for i, count in c.most_common(top)]
    else:
      print("Let's skip this")

def take_step(n, matrix):
    rang = len(matrix[n])
    if np.any(matrix[n]):
        next_n = np.random.choice(range(rang), p=matrix[n])
    else:
        next_n = np.random.choice(range(rang))
    return next_n

In [0]:
%%time
keywords_tr_old = data['content_norm'].apply(lambda x: get_kws_old(x, 10, 10))

Let's skip this
Let's skip this
Let's skip this
Let's skip this
Let's skip this
Let's skip this
Let's skip this
Let's skip this
Let's skip this
Let's skip this
Let's skip this
Let's skip this
Let's skip this
Let's skip this
Let's skip this
CPU times: user 19min 44s, sys: 4.68 s, total: 19min 49s
Wall time: 19min 46s


In [0]:
evaluate(data['keywords'], keywords_tr_old)

Precision -  0.09
Recall -  0.11
F1 -  0.09
Jaccard -  0.05


Опять такой вид алгоритм, где строяися матрицы из грязных текстов не так хорошо работает. Это мне кажется связан с тем, что в текстах много "шума" и перед таким алгоритмом надо было бы их перепредобрабатывать. 